# Calculating the USD Liquidity Index with the OpenBB Platform

This popular indicator is made from a simple subtraction of three FRED series that are published every Wednesday, and is often overlayed with risk assets like the S&P 500 Index or Bitcoin.  The OpenBB SDK is well suited for this task, let's take a look to create this index.

The formula is defined as:

```console
WALCL (All Liabilities) – WLRRAL (RRP) – WDTGAL (TGA)
```

To get these data series, we will use the `openbb-fred` data extension and the `economy` module.  First thing is to import the Python interface, and we will also import Pandas to conduct some DataFrame operations.

In [1]:
from openbb import obb
from pandas import DataFrame


There are two `fred` functions in the `openbb-economy` router:

- `obb.economy.fred_search()`
- `obb.economy.fred_series()`

In [2]:
data = obb.economy.fred_series(["WALCL", "WLRRAL", "WDTGAL", "SP500"])

data

OBBject

id: 0660e135-3511-73d7-8000-b792f7e418bb
results: [{'date': datetime.date(2002, 12, 18), 'provider': 'fred', 'WALCL': 719542...
provider: fred
warnings: None
chart: None
extra: {'results_metadata': {'WALCL': {'title': 'Assets: Total Assets: Total Assets...

There is metadata from each series in the warnings of the response object.  It can be recovered as a JSON dictionary.

In [3]:
metadata = data.extra["results_metadata"]

display(metadata.keys())
display(metadata["WALCL"].get("title"))
display(metadata["WALCL"].get("units"))

dict_keys(['WALCL', 'WLRRAL', 'WDTGAL', 'SP500'])

'Assets: Total Assets: Total Assets (Less Eliminations from Consolidation): Wednesday Level'

'Millions of U.S. Dollars'

## Querying FRED

If we didn't already know the ID for the series, we can search with:

In [13]:
# The first result is the series we are looking for as the starting value.

obb.economy.fred_search("Wednesday Levels").to_df().head(3)

,series_id,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,notes,popularity,group_popularity
0,WALCL,Assets: Total Assets: Total Assets (Less Elimi...,2002-12-18,2023-12-13,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-12-14 15:34:03-06:00,None,95,94
1,H41RESPPALDKNWW,Assets: Liquidity and Credit Facilities: Loans...,2002-12-18,2023-12-13,Weekly,W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-12-14 15:35:03-06:00,None,73,73
2,TREAST,Assets: Securities Held Outright: U.S. Treasur...,2002-12-18,2023-12-13,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-12-14 15:35:02-06:00,The total face value of U.S. Treasury securiti...,72,72


In [14]:
# Adding "Reverse Repo" to the search returns the second series in the equation, as the first result.

obb.economy.fred_search("Wednesday Levels Reverse Repo").to_df().head(3)

,series_id,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,notes,popularity,group_popularity
0,WLRRAL,Liabilities and Capital: Liabilities: Reverse ...,2002-12-18,2023-12-13,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-12-14 15:34:09-06:00,Reverse repurchase agreements are transactions...,64,64
1,WLRRAFOIAL,Liabilities and Capital: Liabilities: Reverse ...,2002-12-18,2023-12-13,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-12-14 15:34:12-06:00,Reverse repurchase agreements are transactions...,40,40
2,WLRRAOL,Liabilities and Capital: Liabilities: Reverse ...,2002-12-18,2023-12-13,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-12-14 15:34:16-06:00,None,24,24


In [17]:
# Refining the search for the Treasury General Account, returns the final series in the equation, as the first result.

obb.economy.fred_search("Wednesday Levels Treasury General").to_df().head(3)

,series_id,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,notes,popularity,group_popularity
0,WDTGAL,Liabilities and Capital: Liabilities: Deposits...,2002-12-18,2023-12-13,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-12-14 15:34:08-06:00,This account is the primary operational accoun...,65,65
1,D2WLTGAL,Liabilities and Capital: Liabilities: Deposits...,2002-12-18,2023-12-13,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-12-14 15:34:10-06:00,None,62,62
2,WLDLCL,Liabilities and Capital: Liabilities: Deposits...,2002-12-18,2023-12-13,"Weekly, As of Wednesday",W,Millions of U.S. Dollars,Mil. of U.S. $,Not Seasonally Adjusted,NSA,2023-12-14 15:34:13-06:00,"This item is the sum of ""Term deposits held by...",23,23


In [22]:
# Several major equity indices are published to FRED, S&P 500 is one of them.

obb.economy.fred_search("SP500").to_df().head(2)

,series_id,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,notes,popularity,group_popularity
0,SP500,S&P 500,2013-12-16,2023-12-15,"Daily, Close",D,Index,Index,Not Seasonally Adjusted,NSA,2023-12-15 19:11:01-06:00,The observations for the S&P 500 represent the...,83,83


By looking at the descriptions, we can confirm that all three Federal Reserve series are numbers as `Millions of USD`.  If they were not all equivalent, some adjustments would need to be made before applying the equation.

In [5]:
for id in metadata:
    display(f"{id}: {metadata[id]['units']}")


'WALCL: Millions of U.S. Dollars'

'WLRRAL: Millions of U.S. Dollars'

'WDTGAL: Millions of U.S. Dollars'

'SP500: Index'

Inspecting the time series element shows that the S&P 500 data (as published to FRED) does not extend as far back as the others.  Let's drop the NaN values and start the time series at a common starting point, which is approximately ten years ago.

In [38]:
display(data.to_df().head(4))
display(data.to_df().dropna().head(4))

# We'll create a new DataFrame object with the dropped rows.
liquidity_index = DataFrame(data.to_df().dropna())

,WALCL,WLRRAL,WDTGAL,SP500
date,,,,
2002-12-18,719542.0,21905.0,6595.0,NaN
2002-12-25,732059.0,20396.0,4662.0,NaN
2003-01-01,730994.0,21091.0,4420.0,NaN
2003-01-08,723762.0,18709.0,5490.0,NaN


,WALCL,WLRRAL,WDTGAL,SP500
date,,,,
2013-12-18,4008062.0,111966.0,116998.0,1810.65
2014-01-08,4028185.0,150726.0,110758.0,1837.49
2014-01-15,4071528.0,156266.0,87926.0,1848.38
2014-01-22,4097914.0,201954.0,96724.0,1844.86


Applying the formula will simply be a matter of subtracting the first, three, columns.

In [49]:

liquidity_index["USD Liquidity Index"] = (liquidity_index["WALCL"] - liquidity_index["WLRRAL"] - liquidity_index["WDTGAL"])

liquidity_index.tail(4)

,WALCL,WLRRAL,WDTGAL,SP500,USD Liquidity Index
date,,,,,
2023-11-22,7810814.0,1271679.0,693007.0,4556.62,5846128.0
2023-11-29,7796145.0,1254156.0,753570.0,4550.58,5788419.0
2023-12-06,7737385.0,1184528.0,667678.0,4549.34,5885179.0
2023-12-13,7739566.0,1152232.0,630618.0,4707.09,5956716.0


Now that there are two items to compare, let's draw it!

In [95]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_scatter(
    x=liquidity_index.index, y=liquidity_index["USD Liquidity Index"]/1000, name="USD Liquidity Index (Billions)", yaxis="y1")

fig.add_scatter(
    x=liquidity_index.index, y=liquidity_index["SP500"], name="S&P 500 Index", yaxis="y2")

fig.update_layout(
    yaxis=dict(title="USD Liquidity Index (Billions)",side="left", position=0, titlefont=dict(size=12), showgrid=False),
    yaxis2=dict(title="S&P 500 Index", side="right", overlaying="y", position=1, titlefont=dict(size=12)),
    title="USD Liquidity Index vs. S&P 500 Index",
    title_y=0.90,
    title_x=0.5,
    autosize=True,
)


To draw them both on the same y-axis, they will need to be normalized.  There are several methods for normalizing a series, the fourth function in the block below paramaterizes a few of them, making it easy to A/B them.

In [85]:
y_axis = liquidity_index[["USD Liquidity Index", "SP500"]]

def absolute_maximum_scale(series):
    return series / series.abs().max()

def min_max_scaling(series):
    return (series - series.min()) / (series.max() - series.min())

def z_score_standardization(series):
    return (series - series.mean()) / series.std()

methods = {"z": z_score_standardization, "m": min_max_scaling, "a": absolute_maximum_scale}

def normalize(data: DataFrame, method: str = "z") -> DataFrame:
    for col in data.columns:
        data.loc[:, col] = methods[f"{method}"](data.loc[:, col])

    return data

normalized = normalize(y_axis, method = "m")

normalized.tail(3)

,USD Liquidity Index,SP500
date,,
2023-11-29,0.721351,0.920274
2023-12-06,0.748489,0.919866
2023-12-13,0.768553,0.971734


Now they can be easily plotted using the built-in `DataFrame.plot` method.

In [91]:
import pandas as pd
pd.options.plotting.backend = "plotly"
normalized.plot()

If you just want to visualize the results, this is a fast way of doing it. However, titles are other customizatons clean things up.

In [94]:
fig = go.Figure()

fig.add_scatter(
    x=normalized.index, y=normalized["USD Liquidity Index"], name="USD Liquidity Index")

fig.add_scatter(
    x=normalized.index, y=normalized["SP500"], name="S&P 500 Index")

fig.update_layout(
    title="USD Liquidity Index vs. S&P 500 Index (Normalized)",
    title_y=0.90,
    title_x=0.5,
    autosize=True,
)

The combinations are endless and we love seeing your creations, tag us on social media with your custom indexes and indicators.